In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Blood_Type/GSE82303'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Blood exsosomal miRNAs of patients with normal livers and liver disease patients"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['subject id: NL30', 'subject id: NL32', 'subject id: NL34', 'subject id: NL35', 'subject id: NL36', 'subject id: NL37', 'subject id: NL38', 'subject id: NL39', 'subject id: NL41', 'subject id: NL42', 'subject id: NL43', 'subject id: NL44', 'subject id: NL45', 'subject id: NL46', 'subject id: NL47', 'subject id: NL48', 'subject id: NL49', 'subject id: NL50', 'subject id: NL51', 'subject id: NL52', 'subject id: NL53', 'subject id: NL54', 'subject id: NL55', 'subject id: NL56', 'subject id: NL58', 'subject id: NL59', 'subject id: NL60', 'subject id: NL61', 'subject id: NL62', 'subject id: NL63'], 1: ['tissue: blood exosome', 'age: 80', 'age: 66', 'age: 39', 'age: 31', 'age: 68', 'age: 34', 'age: 63', 'a

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Explicitly set gene expression data availability for miRNA dataset.
is_gene_available = False

# Check availability of variables in the sample characteristics dictionary

# Blood_Type availability and conversion (not available)
trait_row = None

# Age availability and conversion
for key, values in sample_characteristics_dict.items():
    if any(isinstance(value, str) and 'age' in value.lower() for value in values):
        age_row = key
        break

# Gender availability and conversion
for key, values in sample_characteristics_dict.items():
    if any(isinstance(value, str) and 'gender' in value.lower() for value in values):
        gender_row = key
        break

# Define data type conversion functions
def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    try:
        val = value.split(':')[-1].strip().lower()
        if val == 'female':
            return 0
        elif val == 'male':
            return 1
    except IndexError:
        return None
    return None

# Save cohort information
save_cohort_info('GSE82303', './preprocessed/Blood_Type/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is available (Note: This will always be False here)
if trait_row is not None: 
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Blood_Type', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Blood_Type/trait_data/GSE82303.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
